In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quantum Enhanced Optimization for Radar and Communications Applications 


The Low Autocorrelation Binary Sequences (LABS) is an important and challenging optimization problem with applications related to radar, telecommunications, and other signal related applications. This CUDA-Q Academic module will focus on a clever quantum-enhanced hybrid method developed in a collaboration between Kipu Quantum, University of the Basque Country EHU, and NVIDIA for solving the LABS problem. (This notebook was jointly developed with input from all collaborators.)

Other CUDA-Q Academic modules like [Divide and Conquer MaxCut QAOA](https://github.com/NVIDIA/cuda-q-academic/tree/main/qaoa-for-max-cut) and [Quantum Finance](https://github.com/NVIDIA/cuda-q-academic/blob/main/quantum-applications-to-finance/03_qchop.ipynb), demonstrate how quantum computing can be used outright to solve optimization problems. This notebook demonstrates a slightly different approach. Rather than considering QPUs as the tool to produce the final answer, it demonstrates how quantum can be used to enhance the effectiveness of leading classical methods.  

The benefits of such an approach were highlighted in [Scaling advantage with quantum-enhanced memetic tabu search for LABS](https://arxiv.org/html/2511.04553v1).  This notebook, co-created with the authors of the paper, will allow you to explore the findings of their research and write your own CUDA-Q code that builds a representative quantum-enhanced workflow for solving the LABS problem. Moreover, it will introduce advancements in counteradiabatic optimization techniques on which reduce the quantum resources required to run on a QPU.

**Prerequisites:** This lab assumes you have a basic knowledge of quantum computing, including operators, gates, etc.  For a refresher on some of these topics, explore the [Quick start to Quantum](https://github.com/NVIDIA/cuda-q-academic/tree/main/quick-start-to-quantum) series.

**In this lab you will:**
* 1. Understand the LABS problem and its relation ot radar and communication applications.
* 2. Solve LABS classically with memetic tabu search and learn about the limitations of such methods.
* 3. Code a couteradiabatic algorithm using CUDA-Q to produce approximate solutions to the LABS problem.
* 4. Use the CUDA-Q results to seed your tabu search and understand the potential benefits of this approach.


**Terminology you will use:**
* Low autocorrelation of binary sequences (LABS)
* counteradiabatic optimization
* memetic-tabu search

**CUDA-Q Syntax you will use:**
* cudaq.sample()
* @cudaq.kernel
* ry(), rx(), rz(), x(), h() 
* x.ctrl()

Run the code below to initialize the libraries you will need.

In [1]:
import cudaq
import numpy as np
from math import floor
import auxiliary_files.labs_utils as utils

## The LABS problem and applications

The **Low Autocorrelation Binary Sequences (LABS)** problem is fundamental to many applications, but originated with applications to radar. 

Consider a radar that monitors airport traffic.  The radar signal sent to detect incoming planes must have as much range as possible to ensure safe approaches are planned well in advance.  The range of a radar signal can be increased by sending a longer pulse.  However, in order to differentiate between multiple objects, pulses need to be short to provide high resolution. So, how do you handle situations where you need both?

One solution is a technique called pulse compression.  The idea is to send a long signal, but vary the phase at regular intervals such that the resolution is increased. Generally, the initial signal will encode a binary sequence of phase shifts, where each interval corresponds to a signal with a 0 or 180 degree phase shift. 

The tricky part is selecting an optimal encoding sequence.  When the signal returns, it is fed into a matched filter with the hope that a singular sharp peak will appear, indicating clear detection.  The autocorrelation of the original signal, or how similar the signal is to itself,  determines if a single peak or a messier signal with sidelobes will be detected. A signal should have high autocorrelation when overlayed on top of itself, but low autocorrelation when shifted with a lag. 

Consider the image below.  The signal on the left has a crisp single peak while the single on the right produces many undesirable sidelobes which can inhibit clear detection.  

<img src="images/quantum_enhanced_optimization_LABS/radar.png" width="800">


So, how do you select a good signal?   This is where LABS comes in, defining these questions as a binary optimization problem. Given a binary sequence of length $N$, $(s_1 \cdots s_N) \in {\pm 1}^N$, the goal is to minimize the following objective function.

$$ E(s) = \sum_{k=1}^{N-1} C_k^2 $$

Where $C_k$ is defined as. 

 $$C_k= \sum_{i=1}^{N-k} s_is_{i+k}$$


So, each $C_k$ computes how similar the original signal is to the shifted one for each offset value $k$.  To explore this more, try the interactive widget linked [here](https://nvidia.github.io/cuda-q-academic/interactive_widgets/labs_visualization.html).  See if you can select a very good and very poor sequence and see the difference for the case of $N=7$.

## Classical Solution of the LABS problem

The LABS problem is tricky to solve for a few reasons. First, the configuration space grows exponentially.  Second, underlying symmetries of the problem result in many degeneracies in the optimization landscape severely inhibiting local search methods. 

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 1:</h3>
    <p style="font-size: 16px; color: #333;">
Using the widget above, try to find some of the symmetries for the LABS problem. That is, for a fixed bitstring length, can you find patterns to produce the same energy with different pulse patterns. 
</div>

The best known performance for a classical optimization technique is Memetic Tabu search (MTS) which exhibits a scaling of $O(1.34^N)$.  The MTS algorithm is depicted below.  It begins with a randomly selected population of bitstrings and finds the best solution from them.  Then, a child is selected by sampling directly from or combining multiple bitstrings from the population.  The child is mutated with probability $p_{mutate}$ and then input to a tabu search, which performs a modified greedy local search starting from the child bitstring.  If the result is better than the best in the population, it is updated as the new leader and randomly replaces a  bitstring in the population.


<img src="images/quantum_enhanced_optimization_LABS/mts_algorithm.png" width="500">

Such an approach is fast, parallelizable, and allows for exploration with improved searching of the solution landscape.  

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 2:</h3>
    <p style="font-size: 16px; color: #333;">
Before exploring any quantum approach, get a sense for how MTS works by coding it yourself based generally on the figure above. Algorithms for the combine and mutate steps are provided below as used in the paper. You may need to research more specific details of the process, especially for how tabu search is performed. The MTS procedure should output and optimal bitstring and its energy.  Also, write a function to visualize the results including the energy distribution of the final population.
</div>



<img src="images/quantum_enhanced_optimization_LABS/combine_mutate.png" width="400">



In [2]:
#TODO - Write code to perform MTS

def random_spin_string(N):
    # values ∈ {+1, -1}
    # rng choice: 0 -> +1, 1 -> -1
    bits = np.random.randint(0, 2, size=N, dtype=np.int8)
    s = 1 - 2*bits      # 0 -> 1, 1 -> -1
    return s

def flip_spin(s, i):
    s[i] *= -1          # ±1 flip

def labs_energy_pm1(s):
    s = np.asarray(s, dtype=np.int8)
    N = len(s)
    E = 0
    for k in range(1, N):
        Ck = int(np.dot(s[:N-k], s[k:]))  # sum s_i s_{i+k}
        E += Ck * Ck
    return E

def combine(p1, p2):
    N = len(p1)
    k = np.random.randint(1, N)  # 1..N-1
    return np.concatenate([p1[:k], p2[k:]])

def mutate(s, p_mut):
    s = s.copy()
    for i in range(len(s)):
        if np.random.rand() < p_mut:
            s[i] *= -1          # flip ±1
    return s

s = random_spin_string(10)
print("sequence:", s)
print("unique values:", np.unique(s))
print("energy:", labs_energy_pm1(s))

sequence: [ 1  1  1 -1  1 -1  1  1  1 -1]
unique values: [-1  1]
energy: 37


## Building a Quantum Enhanced Workflow

Despite the effectiveness of MTS, it still exhibits exponential scaling  $O(1.34^N)$ behavior and becomes intractable for large $N$.  Quantum computing provides a potential alternative method for solving the LABS problem because the properties of entanglement, interference, and superpositon may allow for a better global search.  Recent demonstrations have even produced evidence that the quantum approximate optimization algorithm (QAOA) can be used to reduce the scaling of the LABS problem to $O(1.21^N)$ for $N$ between 28 and 40 with quantum minimum finding.

However, current quantum hardware limitations restrict solution to problems of greater than about $N=20$, meaning that it will be some time before quantum approaches can outperform the classical state of the art. It should also be noted that standard QAOA can struggle with LABS and require many layers to converge the parameters if other tricks are not employed.

The authors of [Scaling advantage with quantum-enhanced memetic tabu search for LABS](https://arxiv.org/html/2511.04553v1) cleverly explored an alternate path that combines quantum and classical approaches and might be able to provide a more near-term benefit.  Instead of expecting the quantum computer to solve the problem entirely, they asked how a quantum approach might enhance MTS.

The basic idea is that a quantum optimization routine could run first and the resulting state be sampled to produce a better population for MTS. Many such heuristics for defining the initial population are possible, but the rest of this notebook will explore their methodology, help you to build the workflow yourself, and allow you to analyze the benefits of their approach.

The first step of quantum enhanced MTS (QE-MTS) is to prepare a circuit with CUDA-Q that approximates the ground state of the Hamiltonian corresponding to the LABS problem. You could do this with any optimization algorithm such as QAOA or using an adiabatic approach.  (See the [Quantum Portfolio Optimization](https://github.com/NVIDIA/cuda-q-academic/blob/main/quantum-applications-to-finance/03_qchop.ipynb) CUDA-Q Academic lab for a detailed comparison of these two common approaches.)

The authors of this work opted for an adiabatic approach (More on why later). Recall that the goal of an adiabatic optimization is to begin with a Hamiltonian that has an easily prepared ground state ($H_i$). Then, the adiabatic Hamiltonian $H_{ad}$ can be constructed as $H_{ad}(\lambda) = (1-\lambda)H_i +\lambda H_f $, where $\lambda$ is a function of time and $H_f$ is the Hamiltonian representing a qubit encoding of the LABS problem. 

$$H_f = 2 \sum_{i=1}^{N-2} \sigma_i^z \sum_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} \sigma_{i+k}^z 
+ 4 \sum_{i=1}^{N-3} \sigma_i^z \sum_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \sum_{k=t+1}^{N-i-t} \sigma_{i+t}^z \sigma_{i+k}^z \sigma_{i+k+t}^z$$

The authors also selected $H_i = \sum_i h^x_i \sigma^x_i $ which has an easily prepared ground state of $\ket{+}^{\otimes N}$.

The challenge for implementing the optimization procedure becomes selection of an operator that will quickly and accurately evolve to the ground state of $H_f$.  One approach is to use a so-called auxiliary countradiabatic (CD) term $H_{CD}$, which corrects diabatic transitions that jump out of the ground state during the evolution. The figure below demonstrates the benefit of using a CD correction.


<img src="images/quantum_enhanced_optimization_LABS/counteradiabatic.png" width="900">




An operator called the adiabatic gauge potential $A_{\lambda}$ is the ideal choice for the CD term as it suppresses all possible diabatic transitions, resulting in the following total system to evolve.

$$ H(\lambda) = H_{ad}(\lambda) + \lambda H_{CD} (\lambda) $$

$A(\lambda)$ is derrived from $H_{ad}(\lambda)$  (see paper for details) as it contains information about underlying physics of the problem. 

There is a problem though.  The $A(\lambda)$ term cannot be efficiently expressed exactly and needs to be approximated.  It also turns out that in the so called impulse regime, where the adiabatic evolution is very fast, $H_{cd} (\lambda)$ dominates $H_{ad}(\lambda)$, meaning that the final implementation corresponds to the operator $H(\lambda) = H^1_{cd}(\lambda)$ where  $H^1_{cd}(\lambda)$ is a first order approximation of $A(\lambda)$ (see equation 7 in the paper).

A final step is to use Trotterization to define the quantum circuit to apply $e^{-\theta (t) i H_{cd}}$. The details for this derivation are shown in the appendix of the paper. and result from equation B3 is shown below.  

$$
\begin{equation}
\begin{aligned}
U(0, T) = \prod_{n=1}^{n_{\text{trot}}} & \left[ \prod_{i=1}^{N-2} \prod_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} R_{Y_i Z_{i+k}}\big(4\theta(n\Delta t)h_i^x\big) R_{Z_i Y_{i+k}}\big(4\theta(n\Delta t)h_{i+k}^x\big) \right] \\
& \times \prod_{i=1}^{N-3} \prod_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \prod_{k=t+1}^{N-i-t} \bigg( R_{Y_i Z_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_i^x\big) \\
& \quad \times R_{Z_i Y_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+t}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Y_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Z_{i+k} Y_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k+t}^x\big) \bigg)
\end{aligned}
\end{equation}
$$

It turns out that this implementation is more efficient than QAOA in terms of gate count. The authors calculated that for $N=67$, QAOA would require 1.4 million entangling gates while the CD approach derived here requires only 236 thousand entangling gates.


<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 3:</h3>
    <p style="font-size: 16px; color: #333;">
At first glance, this equation might looks quite complicated. However, observe the structure and note two "blocks" of terms.  Can you spot them?  

They are 2 qubit terms that look like $R_{YZ}(\theta)$ or $R_{ZY}(\theta)$.

As well as 4 qubit terms that look like $R_{YZZZ}(\theta)$, $R_{ZYZZ}(\theta)$, $R_{ZZYZ}(\theta)$, or $R_{ZZZY}(\theta)$.

Thankfully the authors derive a pair of circuit implementations for the two and four qubit terms respectively, shown in the figures below.

Using CUDA-Q, write a kernel for each which will be used later to construct the full implementation.

* Hint: Remember that the adjoint of a rotation gate is the same as rotating in the opposite direction. 

* Hint: You may also want to define a CUDA-Q kernel for an R$_{ZZ}$ gate.

* Hint: Implementing a circuit from a paper is a great place where AI can help accelerate your work.  If you have access to a coding assistant, feel free to use it here.
</div>

<img src="images/quantum_enhanced_optimization_LABS/kernels.png" width="1300">


In [15]:
# TODO  Write CUDA-Q kernels to apply the 2 and 4 qubit operators. 
pi = 3.141592653589793

#2 qubit operators
@cudaq.kernel
def rzz(theta: float, qubits: cudaq.qview):
    q0 = qubits[0]
    q1 = qubits[1]
    x.ctrl(q0, q1)
    rz(theta, q1)
    x.ctrl(q0, q1)

@cudaq.kernel
def ryz(theta: float, qubits: cudaq.qview):
    q0 = qubits[0]
    q1 = qubits[1]

    # Basis change on q0: Y -> Z
    rz(-np.pi / 2, q0)   # S†
    h(q0)

    # Now apply ZZ rotation
    x.ctrl(q0, q1)
    rz(theta, q1)
    x.ctrl(q0, q1)

    # Undo basis change
    h(q0)
    rz(np.pi / 2, q0)    # S

@cudaq.kernel
def rzy(theta: float, qubits: cudaq.qview):
    q0 = qubits[0]
    q1 = qubits[1]

    # Basis change on q1: Y -> Z
    rz(-np.pi / 2, q1)   # S†
    h(q1)

    # ZZ rotation
    x.ctrl(q0, q1)
    rz(theta, q1)
    x.ctrl(q0, q1)

    # Undo basis change
    h(q1)
    rz(np.pi / 2, q1)    # S

#4 qubit operators
@cudaq.kernel
def rzzzz(theta: float, qubits: cudaq.qview):
    q0 = qubits[0]
    q1 = qubits[1]
    q2 = qubits[2]
    q3 = qubits[3]

    # Parity-encode Z⊗Z⊗Z⊗Z into q3, rotate, then uncompute.
    x.ctrl(q0, q3)
    x.ctrl(q1, q3)
    x.ctrl(q2, q3)

    rz(theta, q3)

    x.ctrl(q2, q3)
    x.ctrl(q1, q3)
    x.ctrl(q0, q3)

@cudaq.kernel
def ryzzz(theta: float, qubits: cudaq.qview):
    q0 = qubits[0]

    # Map Y->Z on q0
    rz(-np.pi / 2, q0)
    h(q0)

    # Do ZZZZ rotation
    rzzzz(theta, qubits)

    # Undo mapping
    h(q0)
    rz(np.pi / 2, q0)

@cudaq.kernel
def rzyzz(theta: float, qubits: cudaq.qview):
    q1 = qubits[1]
    rz(-np.pi / 2, q1); h(q1)
    rzzzz(theta, qubits)
    h(q1); rz(np.pi / 2, q1)

@cudaq.kernel
def rzzyz(theta: float, qubits: cudaq.qview):
    q2 = qubits[2]
    rz(-np.pi / 2, q2); h(q2)
    rzzzz(theta, qubits)
    h(q2); rz(np.pi / 2, q2)

@cudaq.kernel
def rzzzy(theta: float, qubits: cudaq.qview):
    q3 = qubits[3]
    rz(-np.pi / 2, q3); h(q3)
    rzzzz(theta, qubits)
    h(q3); rz(np.pi / 2, q3)

@cudaq.kernel
def test_rzz(theta: float):
    q = cudaq.qvector(2)
    # start in |00>
    rzz(theta, q)
    # measure all
    mz(q[0])
    mz(q[1])

@cudaq.kernel
def test_ryzzz(theta: float):
    q = cudaq.qvector(4)
    # start from |0000>, apply R_YZZZ
    ryzzz(theta, q)
    for i in range(4):
        mz(q[i])

@cudaq.kernel
def test_rzyzz(theta: float):
    q = cudaq.qvector(4)
    rzyzz(theta, q)
    for i in range(4):
        mz(q[i])

@cudaq.kernel
def test_rzzyz(theta: float):
    q = cudaq.qvector(4)
    rzzyz(theta, q)
    for i in range(4):
        mz(q[i])

@cudaq.kernel
def test_rzzzy(theta: float):
    q = cudaq.qvector(4)
    rzzzy(theta, q)
    for i in range(4):
        mz(q[i])
    
    

theta = np.pi / 3
samples = cudaq.sample(test_ryzzz, theta)
print(samples)

{ 0000:768 1000:232 }



There are a few additional items we need to consider before completing the final implementation of the entire circuit.  One simplification we can make is that for our problem the $h_i^x$ terms are all 1 and any $h_b^x$ terms are 0, and are only there for generalizations of this model. 

The remaining challenge is derivation of the angles that are used to apply each of the circuits you defined above. These are obtained from two terms $\lambda(t)$ and $\alpha(t)$.  


The $\lambda(t)$ defines an annealing schedule and is generally a Sin function which slowly "turns on" the problem Hamiltonian.  For computing our angles, we need the derivative of $\lambda(t)$.

The $\alpha$ term is a bit trickier and is the solution to a set of differential equations which minimize the distance between $H^1_{CD}(\lambda)$ and $A(\lambda)$.  The result is 

$$\alpha(t) = \frac{-\Gamma_1(t)}{\Gamma_2(t)} $$

Where $\Gamma_1(t)$ and $\Gamma_2(t)$ are defined in equations 16 and 17 of the paper and essentially depend on the structure of the optimization problem.  Curious learners can look at the functions in `labs_utils.py`  to see how these are computed, based on the problem size and specific time step in the Trotter process. 


<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 4:</h3>
    <p style="font-size: 16px; color: #333;">
The `compute_theta` function, called in the following cells, requires all indices of the two and four body terms. These will be used again in our main kernel to apply the respective gates.  Use the products in the formula below to finish the function in the cell below.  Save them as `G2` and `G4` where each is a list of lists of indices defining the two and four term interactions. As you are translating an equation to a set of loops, this is a great opportunity to use an AI coding assistant.

$$
\begin{equation}
\begin{aligned}
U(0, T) = \prod_{n=1}^{n_{\text{trot}}} & \left[ \prod_{i=1}^{N-2} \prod_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} R_{Y_i Z_{i+k}}\big(4\theta(n\Delta t)h_i^x\big) R_{Z_i Y_{i+k}}\big(4\theta(n\Delta t)h_{i+k}^x\big) \right] \\
& \times \prod_{i=1}^{N-3} \prod_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \prod_{k=t+1}^{N-i-t} \bigg( R_{Y_i Z_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_i^x\big) \\
& \quad \times R_{Z_i Y_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+t}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Y_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Z_{i+k} Y_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k+t}^x\big) \bigg)
\end{aligned}
\end{equation}
$$

</div>


In [4]:
def get_interactions(N):
    """
    Generates the interaction sets G2 and G4 based on the loop limits in Eq. 15.
    Returns standard 0-based indices as lists of lists of ints.
    
    Args:
        N (int): Sequence length.
        
    Returns:
        G2: List of lists containing two body term indices
        G4: List of lists containing four body term indices
    """
    
    #TODO - complete the loops below to compute G2 and G4 indicies
    G2 = []
    # 2-body terms: indices (i, i+k)
    for i in range(1, N-1):  
        max_k = (N - i) // 2
        for k in range(1, max_k + 1):
            j = i + k
            # convert to 0-based
            G2.append([i - 1, j - 1])

    G4 = []
    # 4-body terms: indices (i, i+t, i+k, i+k+t)
    for i in range(1, N-2):  
        max_t = (N - i - 1) // 2
        for t in range(1, max_t + 1):
            for k in range(t + 1, N - i - t + 1):  
                indices_1based = [i, i + t, i + k, i + k + t]
                G4.append([idx - 1 for idx in indices_1based])
                
    return G2, G4

G2, G4 = get_interactions(7)
print(len(G2), "two-body terms:", G2[:9])
print(len(G4), "four-body terms:", G4[:13])

9 two-body terms: [[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3], [2, 4], [3, 4], [4, 5]]
13 four-body terms: [[0, 1, 2, 3], [0, 1, 3, 4], [0, 1, 4, 5], [0, 1, 5, 6], [0, 2, 3, 5], [0, 2, 4, 6], [1, 2, 3, 4], [1, 2, 4, 5], [1, 2, 5, 6], [1, 3, 4, 6], [2, 3, 4, 5], [2, 3, 5, 6], [3, 4, 5, 6]]




<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 5:</h3>
    <p style="font-size: 16px; color: #333;">
You are now ready to construct the entire circuit and run the counteradiabatic optimization procedure. The final kernel needs to apply Equation 15 for a specified total evolution time $T$ and the `n_steps` number of Trotter steps.  It must also take as input, the indices for the two and four body terms and the thetas to be applied each step, as these cannot be computed within a CUDA-Q kernel.

The helper function `compute_theta` computes the theta parameters for you, using a few additional functions in accordance with the equations defined in the paper.
</div>


In [6]:
@cudaq.kernel
def trotterized_circuit(N: int, G2: list[list[int]], G4: list[list[int]], steps: int, dt: float, T: float, thetas: list[float]):
    
    reg = cudaq.qvector(N)
    h(reg)
    
    # TODO - write the full kernel to apply the trotterized circuit
    # Trotter steps
    for step in range(steps):
        theta_step = thetas[step]

        #2-body block (RYZ, RZY)
        for term_index in range(len(G2)):
            i = G2[term_index][0]
            j = G2[term_index][1]

            theta_ij = theta_step  # angle for this 2-body interaction

            ryz(theta_ij, [reg[i], reg[j]])
            rzy(theta_ij, [reg[i], reg[j]])

        #  4-body block (YZZZ, ZYZZ, ZZYZ, ZZZY)
        for term_index in range(len(G4)):
            a = G4[term_index][0]
            b = G4[term_index][1]
            c = G4[term_index][2]
            d = G4[term_index][3]

            theta_4 = theta_step

            ryzzz(theta_4, [reg[a], reg[b], reg[c], reg[d]])
            rzyzz(theta_4, [reg[a], reg[b], reg[c], reg[d]])
            rzzyz(theta_4, [reg[a], reg[b], reg[c], reg[d]])
            rzzzy(theta_4, [reg[a], reg[b], reg[c], reg[d]])
    for q in reg:
        mz(q)
            
T= 1.0               # total time
n_steps = 1      
dt = T / n_steps
N = 5
G2, G4 = get_interactions(N)

thetas =[]

for step in range(1, n_steps + 1):
    t = step * dt
    theta_vec = utils.compute_theta(t, dt, T, N, G2, G4)
    thetas.append(theta_vec)

# TODO - Sample your kernel to make sure it works
result = cudaq.sample(trotterized_circuit, N, G2, G4, n_steps,
                      dt, T, thetas,
                      shots_count=1000)

print(result)

{ 00000:33 00001:29 00010:29 00011:30 00100:38 00101:32 00110:30 00111:27 01000:39 01001:43 01010:25 01011:20 01100:26 01101:42 01110:29 01111:30 10000:39 10001:37 10010:24 10011:20 10100:22 10101:36 10110:32 10111:34 11000:43 11001:33 11010:25 11011:28 11100:24 11101:31 11110:37 11111:33 }



## Generating Quantum Enhanced Results

Recall that the point of this lab is to demonstrate the potential benefits of running a quantum subroutine as a preprocessing step for classical optimization of a challenging problem like LABS. you now have all of the tools you need to try this for yourself.

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 6:</h3>
    <p style="font-size: 16px; color: #333;">
Use your CUDA-Q code to prepare an initial population for your memetic search algorithm and see if you can improve the results relative to a random initial population.  If you are running on a CPU, you will need to run smaller problem instances. The code below sets up the problem

</div>


In [20]:
# TODO - write code here to sample from your CUDA-Q kernel and used the results to seed your MTS population
import numpy as np

def tabu_search(start, energy_fn, max_iters=50, tabu_tenure=10):
    """
    Simple tabu search starting from 'start' (±1 array).
    Returns (best_sequence, best_energy).
    """
    N = len(start)
    current = start.copy()
    current_E = energy_fn(current)

    best = current.copy()
    best_E = current_E

    # tabu_list[i] = last iteration when bit i was flipped
    tabu_list = np.full(N, -1, dtype=int)

    for it in range(max_iters):
        best_move_i = None
        best_move_E = None

        # Try flipping each bit
        for i in range(N):
            # aspiration: allow tabu move if it improves global best
            is_tabu = (tabu_list[i] >= 0 and
                       (it - tabu_list[i]) < tabu_tenure)

            candidate = current.copy()
            candidate[i] *= -1
            E = energy_fn(candidate)

            if is_tabu and E >= best_E:
                continue

            if best_move_E is None or E < best_move_E:
                best_move_E = E
                best_move_i = i

        if best_move_i is None:
            break  # no legal move

        # apply best move
        current[best_move_i] *= -1
        current_E = best_move_E
        tabu_list[best_move_i] = it

        if current_E < best_E:
            best_E = current_E
            best = current.copy()

    return best, best_E


def mts_labs_from_population(initial_population,
                             energy_fn,
                             p_mut=0.05,
                             n_generations=200,
                             combine_prob=0.7,
                             tabu_iters=50,
                             tabu_tenure=10):
    """
    Run MTS given an INITIAL POPULATION (list of ±1 arrays).
    Returns (best_sequence, best_energy).
    """
    population = [np.array(s, dtype=np.int8).copy()
                  for s in initial_population]
    pop_size = len(population)
    N = len(population[0])

    energies = [energy_fn(ind) for ind in population]
    best_idx = int(np.argmin(energies))
    best = population[best_idx].copy()
    best_E = energies[best_idx]

    for gen in range(n_generations):
        # --- make child ---
        if np.random.rand() < combine_prob and pop_size >= 2:
            i, j = np.random.randint(0, pop_size, size=2)
            child = combine(population[i], population[j])
        else:
            i = np.random.randint(0, pop_size)
            child = population[i].copy()

        # --- mutate + tabu search ---
        child = mutate(child, p_mut)
        child_best, child_E = tabu_search(child, energy_fn,
                                          max_iters=tabu_iters,
                                          tabu_tenure=tabu_tenure)

        # --- update global best ---
        if child_E < best_E:
            best_E = child_E
            best = child_best.copy()

        # --- update population (replace random member) ---
        replace_idx = np.random.randint(0, pop_size)
        population[replace_idx] = child_best
        energies[replace_idx] = child_E

    return best, best_E

def bitstring_to_pm1(bitstr: str):
    # 0 -> +1, 1 -> -1
    bits = np.array([int(c) for c in bitstr], dtype=np.int8)
    return 1 - 2 * bits

def quantum_seed_population(pop_size: int,
                            N: int,
                            G2, G4,
                            n_steps: int,
                            dt: float,
                            T: float,
                            thetas):
    """
    Use trotterized_circuit to generate an initial population of ±1 sequences.
    We oversample and keep the lowest-energy samples.
    """
    # oversample to have choice
    shots = pop_size * 8

    counts = cudaq.sample(
        trotterized_circuit,
        N, G2, G4, n_steps, dt, T, thetas,
        shots_count=shots
    )

    # collect candidates with energies
    candidates = []
    for bitstr, count in counts.items():
        s = bitstring_to_pm1(bitstr)
        E = labs_energy_pm1(s)
        for _ in range(count):
            candidates.append((E, s.copy()))

    # sort by energy (low first)
    candidates.sort(key=lambda x: x[0])

    # take best pop_size
    population = []
    for E, s in candidates:
        population.append(s.copy())
        if len(population) >= pop_size:
            break

    # if somehow not enough, pad with random
    while len(population) < pop_size:
        population.append(random_spin_string(N))

    return population

# Problem size
N = 13
G2, G4 = get_interactions(N)

# Trotter / CD parameters (you can tweak)
T = 1.0
n_steps = 1
dt = T / n_steps

# theta per step (your compute_theta from the lab)
thetas = []
for step in range(1, n_steps + 1):
    t = step * dt
    theta_val = utils.compute_theta(t, dt, T, N, G2, G4)
    thetas.append(theta_val)

pop_size = 40

import time
# build random initial population
rand_population = [random_spin_string(N) for _ in range(pop_size)]
start_rand = time.time()
best_rand, best_rand_E = mts_labs_from_population(
    rand_population,
    labs_energy_pm1,
    p_mut=0.05,
    n_generations=200
)
end_rand = time.time()
runtime_rand = end_rand - start_rand

print("Random initial population best energy:", best_rand_E)
print("Random init runtime (sec):", runtime_rand)
q_population = quantum_seed_population(
    pop_size, N, G2, G4, n_steps, dt, T, thetas
)
start_rand = time.time()
best_q, best_q_E = mts_labs_from_population(
    q_population,
    labs_energy_pm1,
    p_mut=0.05,
    n_generations=200
)
end_rand = time.time()
runtime_rand = end_rand - start_rand
print("Quantum-seeded population best energy:", best_q_E)
print("Quantum-seeded runtime (sec):", runtime_rand)


Random initial population best energy: 6
Random init runtime (sec): 2.5581047534942627
Quantum-seeded population best energy: 6
Quantum-seeded runtime (sec): 2.5081136226654053


The results clearly show that a population sampled from CUDA-Q results in an improved distribution and a lower energy final result. This is exactly the goal of quantum enhanced optimization.  To not necessarily solve the problem, but improve the effectiveness of state-of-the-art classical approaches. 

A few major caveats need to be mentioned here. First, We are comparing a quantum generated population to a random sample.  It is quite likely that other classical or quantum heuristics could be used to produce an initial population that might even beat the counteradiabatic method you used, so we cannot make any claims that this is the best. 

Recall that the point of the counteradiabatic approach derived in the paper is that it is more efficient in terms of two-qubit gates relative to QAOA. The benefits of this regime would only truly come into play in a setting (e.g. larger problem instance) where it is too difficult to produce a good initial population with any know classical heuristic, and the counteradiabatic approach is more efficiently run on a QPU compared to alternatives.

We should also note that we are comparing a single sample of each approach.  Maybe the quantum sample got lucky or the randomly generated population was unlucky and a more rigorous comparison would need to repeat the analysis many times to draw any confidently conclusions.  

The authors of the paper discuss all of these considerations, but propose an analysis that is quite interesting related to the scaling of the technique. Rather than run large simulations ourselves, examine their results below. 


<img src="images/quantum_enhanced_optimization_LABS/tabu_search_results.png" width="900">

The authors computed replicate median (median of solving the problem repeated with same setup) time to solutions (excluding time to sample from QPU) for problem sizes $N=27$ to $N=37$. Two interesting conclusions can be drawn from this. First, standard memetic tabu search (MTS) is generally faster than quantum enhanced (QE) MTS.  But there are two promising trends. For larger problems, the QE-MTS experiments occasionally have excellent performance with times to solution much smaller than all of the MTS data points.  These outliers indicate there are certain instances where QE-MTS could provide much faster time-to-solution. 

More importantly, if a line of best fit is calculated using the median of each set of medians, the slope of the QE-MTS line is smaller than the MTS!  This seems to indicate that QE solution of this problem scales $O(1.24^N)$ which is better than the best known classical heuristic ($O(1.34^N)$) and the best known quantum approach (QAOA - $O(1.46^N)$).

For problems of size of $N=47$ or greater, the authors anticipate that QE-MTS could be a promising technique and produce good initial populations that are difficult to obtain classically. 

The study reinforces the potential of hybrid workflows enhanced by quantum data such that a classical routine is still the primary solver, but quantum computers make it much more effective.  Future work can explore improvements to both the quantum and classical sides, such as including GPU accelerated memetic search on the classical side.

## Self-validation To Be Completed for Phase 1

In this section, explain how you verified your results. Did you calculate solutions by hand for small N? Did you create unit tests? Did you cross-reference your Quantum energy values against your Classical MTS results? Did you check known symmetries?

In [29]:
import numpy as np

def brute_force_min_energy(N):
    best_E = None
    best_s = None
    for mask in range(2 ** N):
        bits = [(mask >> i) & 1 for i in range(N)]
        s = 1 - 2 * np.array(bits, dtype=np.int8)   # 0->+1, 1->-1
        E = labs_energy_pm1(s)
        if best_E is None or E < best_E:
            best_E = E
            best_s = s.copy()
    return best_s, best_E

for N_test in [5, 6, 10]:
    s_star, E_star = brute_force_min_energy(N_test)
    print(f"N = {N_test}, brute-force min E =", E_star)

def run_mts_once(N, pop_size=20):
    init_pop = [random_spin_string(N) for _ in range(pop_size)]
    best, best_E = mts_labs_from_population(
        init_pop,         
        labs_energy_pm1, 
        p_mut=0.05,
        n_generations=200
    )
    return best, best_E


for N_test in [5, 6, 10]:
    _, E_star = brute_force_min_energy(N_test)
    _, E_mts  = run_mts_once(N_test)
    print(f"N = {N_test}, brute-force E = {E_star}, MTS E = {E_mts}")

# symmetry
for _ in range(5):
    s = random_spin_string(10)
    E      = labs_energy_pm1(s)
    E_flip = labs_energy_pm1(-s)
    E_rev  = labs_energy_pm1(s[::-1])

    print("E =", E, "E_flip =", E_flip, "E_rev =", E_rev)

def bitstring_to_pm1(bitstr: str):
    bits = np.array([int(c) for c in bitstr], dtype=np.int8)
    return 1 - 2 * bits  # 0->+1, 1->-1

# small N
N_check = 7
G2_check, G4_check = get_interactions(N_check)
T_check = 1.0
n_steps_check = 1
dt_check = T_check / n_steps_check

thetas_check = []
for step in range(1, n_steps_check + 1):
    t = step * dt_check
    theta_val = utils.compute_theta(t, dt_check, T_check, N_check, G2_check, G4_check)
    thetas_check.append(theta_val)

counts = cudaq.sample(
    trotterized_circuit,
    N_check, G2_check, G4_check,
    n_steps_check, dt_check, T_check, thetas_check,
    shots_count=50
)

print("Some CUDA-Q samples and their classical LABS energies:")
for bitstr, count in list(counts.items())[:5]:
    s = bitstring_to_pm1(bitstr)
    E = labs_energy_pm1(s)
    print(f"bitstring = {bitstr}, E = {E}, count = {count}")

N_demo = 10
G2_demo, G4_demo = get_interactions(N_demo)
T_demo = 1.0
n_steps_demo = 1
dt_demo = T_demo / n_steps_demo

thetas_demo = []
for step in range(1, n_steps_demo + 1):
    t = step * dt_demo
    thetas_demo.append(utils.compute_theta(t, dt_demo, T_demo, N_demo, G2_demo, G4_demo))

# random seeds
rand_pop = [random_spin_string(N_demo) for _ in range(20)]
rand_Es = [labs_energy_pm1(s) for s in rand_pop]

# quantum seeds
q_pop = quantum_seed_population(20, N_demo, G2_demo, G4_demo,
                                n_steps_demo, dt_demo, T_demo, thetas_demo)
q_Es = [labs_energy_pm1(s) for s in q_pop]

print("Random seed energies:", rand_Es)
print("Quantum seed energies:", q_Es)
print("avg random E:", sum(rand_Es)/len(rand_Es))
print("avg quantum E:", sum(q_Es)/len(q_Es))


N = 5, brute-force min E = 2
N = 6, brute-force min E = 7
N = 10, brute-force min E = 13
N = 5, brute-force E = 2, MTS E = 2
N = 6, brute-force E = 7, MTS E = 7
N = 10, brute-force E = 13, MTS E = 13
E = 37 E_flip = 37 E_rev = 37
E = 37 E_flip = 37 E_rev = 37
E = 29 E_flip = 29 E_rev = 29
E = 37 E_flip = 37 E_rev = 37
E = 45 E_flip = 45 E_rev = 45
Some CUDA-Q samples and their classical LABS energies:
bitstring = 1001000, E = 7, count = 1
bitstring = 0100100, E = 23, count = 1
bitstring = 1100111, E = 23, count = 1
bitstring = 1000110, E = 19, count = 2
bitstring = 1000000, E = 31, count = 1
Random seed energies: [37, 13, 21, 29, 45, 37, 37, 37, 37, 21, 21, 29, 45, 21, 37, 29, 21, 109, 45, 29]
Quantum seed energies: [13, 13, 13, 13, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21]
avg random E: 35.0
avg quantum E: 19.4
